In [14]:
#IMPORT USED PACKAGES AND SET SEED
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import *
import pickle
import random
from datetime import datetime
import pandas as pd
import csv
import pbs
import os
import sys
import itertools
import json
from multiprocessing import Pool
from spellchecker import SpellChecker
from IPython.display import display
random.seed(123)

In [15]:
time = datetime.strftime(datetime.now(), "%Y_%m_%d_%H_%M_%S")

# Preprocessing

#### Remove duplicated tweets

In [16]:
def drop_duplicate(input_filename):
    with open("Datasets/"+input_filename, encoding="utf-8") as f :
        tweets = f.read().splitlines()
        df_tweets = pd.DataFrame(tweets,columns=['Tweets'])
        df_tweets.drop_duplicates(inplace=True)
    with open('Processed_data/no_dupl_' + input_filename, "w+", encoding = "UTF-8") as f:
        for tweet in df_tweets["Tweets"]:
            f.write("%s\n" % tweet)

In [17]:
for file in ['train_pos.txt','train_neg.txt','train_neg_full.txt','train_pos_full.txt']:
    drop_duplicate(file)

In [18]:
# drop duplicates test
with open("Datasets/test_data.txt", encoding="utf-8") as f :
    test = f.read().splitlines()
    tweets = []
    ids = []
    for line in test:
        id, tweet = line.split(',',1)
        tweets.append(tweet)
        ids.append(id)
    df_test = pd.DataFrame(tweets,ids,columns=['Tweets'])
    df_test.drop_duplicates(inplace=True)
    
with open('Processed_data/no_dupl_test_data.txt', "w+", encoding = "UTF-8") as f:
    for tweet in df_test["Tweets"]:
        f.write("%s\n" % tweet)

#### Concatenate all text files (pos, neg and test) to create a dataset used as reference for twitter language

In [19]:
with open("Processed_data/twitter_language.txt", "w+", encoding = "utf-8") as f:
    for path in ['Processed_data/no_dupl_train_pos.txt','Processed_data/no_dupl_train_neg.txt', 'Processed_data/no_dupl_test_data.txt']:
        with open(path,encoding="utf-8") as file:
            f.write(file.read())

In [20]:
with open("Processed_data/twitter_language_full.txt", "w+", encoding = "utf-8") as f:
    for path in ['Processed_data/no_dupl_train_pos_full.txt','Processed_data/no_dupl_train_neg_full.txt', 'Processed_data/no_dupl_test_data.txt']:
        with open(path,encoding="utf-8") as file:
            f.write(file.read())

###  Spell correction

In [21]:
## Create a frequency dictionnary of words occuring at least *threshold* times in the dataset, 
## that will be used as a reference for spell checking

output_filename = 'Processed_data/vocab_freq.txt'
vocab_successful = os.system("build_vocab.sh Processed_data/twitter_language.txt " + output_filename)

if (vocab_successful != 0):
    sys.exit("Building vocabulary failed.")

In [22]:
## Create a frequency dictionnary of words occuring at least *threshold* times in the dataset, 
## that will be used as a reference for spell checking

output_filename = 'Processed_data/vocab_freq_full.txt'
vocab_successful = os.system("build_vocab.sh Processed_data/twitter_language_full.txt " + output_filename)

if (vocab_successful != 0):
    sys.exit("Building vocabulary failed.")

In [23]:
def build_dictionnary(input_vocab, threshold):
    full_vocab = pd.read_csv(input_vocab+".txt", sep = "\s+", header=None, engine='python')
    cut_vocab = full_vocab[full_vocab[0] >= threshold]
    cut_vocab.columns = ["number occ","word"]
    cut_vocab.set_index("word",inplace=True)
    with open(input_vocab + '.json', 'w') as f:
        json.dump(cut_vocab["number occ"].to_dict(), f) 

build_dictionnary("Processed_data/vocab_freq", threshold=5) #TODO try other threshold
build_dictionnary("Processed_data/vocab_freq_full", threshold=5) #TODO try other threshold

In [24]:
## Spell correction functions
def correct(tweet, dict_corr):
    list_words = tweet.split()
    for i, word in enumerate(list_words):
        if word in dict_corr :
            list_words[i] = dict_corr[word]
    corr_tweet = ' '.join(list_words)
    return (corr_tweet) 

def spell_correction(data):
    spell = SpellChecker(distance=1) # TODO if possible, try distance=2
    spell.word_frequency.load_dictionary('Processed_data/vocab_freq.json') #'Processed_data/vocab_freq.json')
    dict_corr = {}
    for tweet in data["Tweets"]:
        list_words = tweet.split()
        for i, word in enumerate(list_words):
            if word not in dict_corr :
                if word in spell.unknown([word]):
                    dict_corr[word] = spell.correction(word)
    data["Tweets"] = data["Tweets"].apply(lambda x : correct(x, dict_corr))
    return data, dict_corr

In [25]:
#Define the used datasets here
positive_dataset = 'Processed_data/no_dupl_train_pos.txt' #'Processed_data/no_dupl_train_pos.txt'
negative_dataset = 'Processed_data/no_dupl_train_neg.txt' #'Processed_data/no_dupl_train_neg.txt'

def load_in_pd(data_path):
    with open(data_path, encoding="utf-8") as f :
        x = f.read().splitlines()
        x = pd.DataFrame(x,columns=['Tweets'])
    return (x)

positive_pd = load_in_pd(positive_dataset)
negative_pd = load_in_pd(negative_dataset)

In [26]:
%%time
### APPLYING DESIRED PREPROCESSING (to pos and neg)

#Define the desired preprocessing method in this function
def preprocess(tweet_data):
    tweet_data, dict_corr = spell_correction(tweet_data)
    return tweet_data, dict_corr

positive_preprocessed, dict_corr1 = preprocess(positive_pd)  
with open('Processed_data/' + 'positive_spell.txt', "w+", encoding = "UTF-8") as f: #positive_spell.txt', "w+", encoding = "UTF-8") as f:
    for tweet in positive_preprocessed["Tweets"]:
        f.write("%s\n" % tweet)
del positive_preprocessed
print("done positive")

negative_preprocessed, dict_corr2 = preprocess(negative_pd)
with open('Processed_data/' + 'negative_spell.txt', "w+", encoding = "UTF-8") as f: #negative_spell.txt', "w+", encoding = "UTF-8") as f:
    for tweet in negative_preprocessed["Tweets"]:
        f.write("%s\n" % tweet)
del negative_preprocessed

done positive
Wall time: 1min 27s


In [27]:
### APPLYING DESIRED PREPROCESSING (to test)

test_dataset = "Datasets/test_data.txt"

def load_in_pd_test(data_path):
    with open(data_path, encoding="utf-8") as f :
        test = f.read().splitlines()
        tweets = []
        ids = []
        for line in test:
            id, tweet = line.split(',',1)
            tweets.append(tweet)
            ids.append(id)
        test = pd.DataFrame(tweets,ids,columns=['Tweets'])
    return test

test_pd = load_in_pd_test(test_dataset)
test_spell, dict_corr_test = spell_correction(test_pd) 

with open("Processed_data/test_spell.txt", "w+", encoding="utf-8") as f:
    for index, row in test_spell.iterrows():
        f.write(index + "," + row["Tweets"] + "\n")

In [28]:
# gather input for word embeddings
with open("Processed_data/input_WE.txt", "w+", encoding = "utf-8") as f:
    with open("Processed_data/test_spell.txt",encoding="utf-8") as test:
        test = test.readlines()
        for line in test:
            id, tweet = line.split(',',1)
            f.write("%s" % tweet)
    
    for path in ['Processed_data/positive_spell.txt','Processed_data/negative_spell.txt']:
        with open(path,encoding="utf-8") as file:
            f.write(file.read())

In [ ]:
##APPLY build_vocab.sh AND CUT THE VOCABULARY USING THE CHOSEN THRESHOLD

#Choose the desired cutting parameter here (Tokens with >= cut_threshold occurrences are kept)
cut_threshold = 5


arg1 = 'positive_preprocessed' + time + '.txt'
arg2 = 'negative_preprocessed' + time + '.txt'
arg3 = 'vocab_' + time + '.txt'
vocab_successful = os.system("build_vocab.sh " + arg1 + " " + arg2 + " " + arg3)

if (vocab_successful != 0):
    sys.exit("Building vocabulary failed.")
    
def cut_and_save_vocab(file_in, file_out):
    full_vocab = pd.read_csv(file_in, sep = "(\s+)", header=None, engine = 'python')
    cutted_vocab = full_vocab[full_vocab[0] >= cut_threshold][2]
    with open(file_out, 'w+') as f:
        f.write(cutted_vocab.to_string(header = False, index = False))
    
cut_and_save_vocab('Processed_data/vocab_' + time + '.txt', 'Processed_data/vocab_cut' + time + '.txt')
    

In [ ]:
##DUMP THE BUILT VOCABULARY TO A PICKLE FILE
vocab = dict()
with open('Processed_data/vocab_cut' + time + '.txt') as f:
    for idx, line in enumerate(f):
        vocab[line.strip()] = idx

with open('Processed_data/vocab_' + time + '.pkl', 'wb') as f:
    pickle.dump(vocab, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
##CREATE A CO-OCCURRENCE MATRIX
def create_cooc(vocab_file, negative_file, positive_file, output_file):
    with open(vocab_file, 'rb') as f:
        vocab = pickle.load(f)
    vocab_size = len(vocab)
    data, row, col = [], [], []
    counter = 1
    for fn in [negative_file, positive_file]:
        with open(fn,encoding="utf8") as f:
            for line in f:
                tokens = [vocab.get(t, -1) for t in line.strip().split()]
                tokens = [t for t in tokens if t >= 0]
                for t in tokens:
                    for t2 in tokens:
                        data.append(1)
                        row.append(t)
                        col.append(t2)

                if counter % 10000 == 0:
                    print(counter)
                counter += 1

                if counter % 200000 == 0:
                    print(len(data))
                    cooc = coo_matrix((data, (row, col)))
                    print("summing duplicates (this can take a while)")
                    cooc.sum_duplicates()
                    data=list(cooc.data)
                    row=list(cooc.row)
                    col=list(cooc.col)
                    print(len(data))

    print(len(data))
    cooc = coo_matrix((data, (row, col)))
    print("summing duplicates (this can take a while)")
    cooc.sum_duplicates()
    
    with open(output_file, 'wb') as f:
        pickle.dump(cooc, f, pickle.HIGHEST_PROTOCOL)
        
#Used datasets should be defined on the second code block
create_cooc('Processed_data/vocab_' + time + '.pkl', 'Processed_data/negative_preprocessed' + time + '.txt', 'Processed_data/positive_preprocessed' + time + '.txt',
            'Processed_data/cooc_pickle' + time + '.pkl')

In [ ]:
## APPLY glove.py
random.seed(123)
def glove(cooc_pickle, output_file):
    print("loading cooccurrence matrix")
    with open(cooc_pickle, 'rb') as f:
        cooc = pickle.load(f)
    print("{} nonzero entries".format(cooc.nnz))

    nmax = 100
    print("using nmax =", nmax, ", cooc.max() =", cooc.max())

    print("initializing embeddings")
    embedding_dim = 20
    xs = np.random.normal(size=(cooc.shape[0], embedding_dim))
    ys = np.random.normal(size=(cooc.shape[1], embedding_dim))

    eta = 0.001
    alpha = 3 / 4

    epochs = 10

    for epoch in range(epochs):
        print("epoch {}".format(epoch))
        for ix, jy, n in zip(cooc.row, cooc.col, cooc.data):
            logn = np.log(n)
            fn = min(1.0, (n / nmax) ** alpha)
            x, y = xs[ix, :], ys[jy, :]
            scale = 2 * eta * fn * (logn - np.dot(x, y))
            xs[ix, :] += scale * y
            ys[jy, :] += scale * x
    np.save(output_file, xs)

glove('Processed_data/cooc_pickle' + time + '.pkl', 'Processed_data/embeddings' + time + '.npy')

In [ ]:
##### PREPARE DATA FOR TRAINING A CLASSIFIER

def load_train_data(data_path):
    """Loads data and returns ids (event ids) and X (feature vector)"""
    f = open(data_path, encoding="utf-8")
    x = f.read().splitlines()
    x = pd.DataFrame(x, columns=['Tweets'])
    return x

def representation(tweet, we, vocab):
    acc = np.array(0)
    n_ignored_word = 0
    for word in tweet.split():
        if word not in vocab.keys():
            n_ignored_word += 1
        else:
            try:
                acc = np.add(acc,we[vocab[word]])
            except: 
                #print("problem with " + word) #last word from vocab is missing in cooc
                n_ignored_word += 1
    n = len(tweet.split()) - n_ignored_word
    acc = acc / n
    return(acc)


def create_train_data(positive_path, negative_path, vocab, we):
    pos = load_train_data(positive_path)
    neg = load_train_data(negative_path)
    pos["y"] = 1
    neg["y"] = -1

    print('pos...')
    
    pos.reset_index(drop = True, inplace = True)
    pos["w"] = pos["Tweets"].apply(lambda x: representation(x, we, vocab))
    pos.drop("Tweets", axis=1, inplace = True)

    col = ["w" + str(k) for k in range(np.shape(we)[1])]
    pos[col] = pos["w"].apply(pd.Series)
    pos.drop("w",axis=1,inplace=True)
    
    #remove the tweets which do not have any words used more than 5 times in the training dataset
    pos.dropna(inplace=True) 
    
    
    with open('Processed_data/pos' + time + '.pkl', 'wb') as k:
        pickle.dump(pos, k, pickle.HIGHEST_PROTOCOL)
    del pos
    print('pos done')
    
    print('neg')
    neg.reset_index(drop = True, inplace = True)
    neg["w"] = neg["Tweets"].apply(lambda x: representation(x, we, vocab))
    neg.drop("Tweets", axis=1, inplace = True)

    col = ["w" + str(k) for k in range(np.shape(we)[1])]
    neg[col] = neg["w"].apply(pd.Series)
    neg.drop("w",axis=1,inplace=True)
    
    #remove the tweets which do not have any words used more than 5 times in the training dataset
    neg.dropna(inplace=True)
    
    print('neg done')
    
    with open('Processed_data/pos' + time + '.pkl', 'rb') as j:
        pos = pickle.load(j)
    
    print('lets try to concatenate')
    train = pd.concat([pos, neg])
    print('tadaaaa')
    return train

with open('Processed_data/vocab_' + time + '.pkl', 'rb') as f:
    vocab = pickle.load(f)
we = np.load('Processed_data/embeddings' + time + '.npy')
#The names of the datasets are defined in the second code block
train = create_train_data(positive_dataset, negative_dataset, vocab, we)

In [ ]:
##TRAIN A CLASSIFIER

from sklearn.linear_model import SGDClassifier
X = train.drop("y", axis=1)
y = train["y"]
random.seed(123)
clf = SGDClassifier()
clf.fit(X, y) 

In [ ]:
##### LOAD THE TEST DATA, USE THE MODEL TO PREDICT THE SENTIMENTS AND SAVE THE PREDICTIONS

def load_and_prepare_test_data(data_path,vocab, we):
    """Loads data and returns ids (event ids) and X (feature vector)"""
    f = open(data_path, encoding='utf-8')
    test = f.read().splitlines()
    tweets = []
    ids = []
    for line in test:
        id, tweet = line.split(',',1)
        tweets.append(tweet)
        ids.append(id)
    test = pd.DataFrame(tweets,ids,columns=['Tweets'])
    test["w"] = test["Tweets"].apply(lambda x: representation(x, we, vocab))
    col = ["w" + str(k) for k in range(np.shape(we)[1])]
    test[col] = test["w"].apply(pd.Series)
    test.drop("w", axis=1, inplace = True)
    test.drop("Tweets", axis=1, inplace = True)
    
    drops = test[test.isnull().any(axis=1)].index
    #for the tweets which do not have any words in the cutted vocabulary, predict 1
    test.fillna(1, inplace=True)
    
    return test,drops

test, drops = load_and_prepare_test_data('Datasets/test_data.txt', vocab, we)
test["Prediction"] = clf.predict(test)
#test["Id"] = test.index
test["Prediction"].to_csv("Submissions/submission" + time + ".csv", header= True, index_label = "Id")